In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random,pickle

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.data import datasets
from detectron2 import model_zoo

In [2]:
import torch 
print(torch.__version__)

1.10.0


In [3]:
torch.cuda.is_available()

True

In [4]:
def cv2_imshow(im):
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.figure(), plt.imshow(im), plt.axis('off');

In [5]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from matplotlib.pyplot import imshow
from PIL import Image
import IPython
def cv2_imshow(img):
    img = img[:,:,[2,1,0]]
    img = Image.fromarray(img)
    plt.figure(figsize=(20, 20))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [6]:
train_dataset_name = 'signs'
train_image_path = "/Users/Rekha/Documents/road_signs/images/train"
train_json_annot_path = "/Users/Rekha/Documents/road_signs/labels/train/coco.json"
f = open(train_json_annot_path, 'r')
train_annotations = json.load(f)
f.close()
num_classes = len(train_annotations['categories'])


In [7]:
test_dataset_name = 'signs_test'
test_image_path = '/Users/Rekha/Documents/road_signs/images/valid'
test_json_annot_path = "/Users/Rekha/Documents/road_signs/labels/valid/coco.json"
f = open(train_json_annot_path, 'r')
train_annotations = json.load(f)
f.close()
num_classes = len(train_annotations['categories'])

In [8]:
cfg_save_path =  '/Users/Rekha/Documents/road_signs/detectron2/IS_cfg.pickle'
training_dict = datasets.load_coco_json(train_json_annot_path, train_image_path,
                dataset_name=train_dataset_name)

[12/15 09:21:24 d2.data.datasets.coco]: Loaded 1484 images in COCO format from /Users/Rekha/Documents/road_signs/labels/train/coco.json


In [9]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances(train_dataset_name, {}, train_json_annot_path, train_image_path)
register_coco_instances(test_dataset_name, {}, test_json_annot_path, test_image_path)

In [10]:
signs_metadata = MetadataCatalog.get("signs")
dataset_dicts = DatasetCatalog.get("signs")

[12/15 09:21:34 d2.data.datasets.coco]: Loaded 1484 images in COCO format from /Users/Rekha/Documents/road_signs/labels/train/coco.json


In [11]:
print(type(signs_metadata))
MetadataCatalog.get("signs")

<class 'detectron2.data.catalog.Metadata'>


namespace(name='signs',
          thing_classes=['red_square',
                         'blue_square',
                         'white_square',
                         'right_of_way',
                         'direction',
                         'blue_circle',
                         'yellow_black_marker',
                         'yield',
                         'zone_start',
                         'red_circle',
                         'red_blue_circle',
                         'yield_bakside',
                         'blue_white_square',
                         'zone_end',
                         'red_triangle'],
          thing_dataset_id_to_contiguous_id={1: 0,
                                             2: 1,
                                             3: 2,
                                             4: 3,
                                             5: 4,
                                             6: 5,
                                             7: 6,
         

In [1]:
#import random

#for d in random.sample(dataset_dicts,10):
    #img = cv2.imread(d["file_name"])
    #visualizer = Visualizer(img[:, :, ::-1], metadata=signs_metadata, scale=0.5)
    #vis = visualizer.draw_dataset_dict(d)
    #cv2_imshow(vis.get_image()[:, :, ::-1])

In [15]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
device = 'cuda'

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"))

cfg.DATASETS.TRAIN = ("signs",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 0
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  
cfg.SOLVER.MAX_ITER = 40000    
cfg.SOLVER.STEPS = [30000]        

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 15 

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[12/15 09:23:38 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_a54504.pkl: 178MB [00:21, 8.13MB/s]                                                                        
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (16, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (16,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (60, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (60,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.

[12/15 09:24:00 d2.engine.train_loop]: Starting training from iteration 0


C:\Users\Rekha\anaconda3\envs\detect\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/15 09:24:15 d2.utils.events]:  eta: 3:22:20  iter: 19  total_loss: 3.54  loss_cls: 2.606  loss_box_reg: 0.05411  loss_mask: 0.694  loss_rpn_cls: 0.0953  loss_rpn_loc: 0.08295  time: 0.3032  data_time: 0.1143  lr: 4.9953e-06  max_mem: 1836M
[12/15 09:24:21 d2.utils.events]:  eta: 3:25:19  iter: 39  total_loss: 3.356  loss_cls: 2.421  loss_box_reg: 0.05709  loss_mask: 0.6938  loss_rpn_cls: 0.06057  loss_rpn_loc: 0.06849  time: 0.3086  data_time: 0.1161  lr: 9.9902e-06  max_mem: 1836M
[12/15 09:24:27 d2.utils.events]:  eta: 3:25:19  iter: 59  total_loss: 2.902  loss_cls: 2.016  loss_box_reg: 0.04468  loss_mask: 0.6913  loss_rpn_cls: 0.05561  loss_rpn_loc: 0.05998  time: 0.3092  data_time: 0.1153  lr: 1.4985e-05  max_mem: 1836M
[12/15 09:24:33 d2.utils.events]:  eta: 3:25:07  iter: 79  total_loss: 2.42  loss_cls: 1.472  loss_box_reg: 0.06348  loss_mask: 0.6878  loss_rpn_cls: 0.07026  loss_rpn_loc: 0.08667  time: 0.3079  data_time: 0.1143  lr: 1.998e-05  max_mem: 1836M
[12/15 09:24:39 d

In [16]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [21]:
signs_test_metadata = MetadataCatalog.get("signs_test")
dataset_dicts = DatasetCatalog.get("signs_test")

[12/15 13:24:06 d2.data.datasets.coco]: Loaded 384 images in COCO format from /Users/Rekha/Documents/road_signs/labels/valid/coco.json


In [19]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("signs_test", )
predictor = DefaultPredictor(cfg)

In [2]:
#from detectron2.utils.visualizer import ColorMode
#dataset_dicts = DatasetCatalog.get("signs_test")
#for d in random.sample(dataset_dicts, 20):    
   # im = cv2.imread(d["file_name"])
    #outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    #v = Visualizer(im[:, :, ::-1],
                  # metadata=signs_test_metadata, 
                  # scale=0.5, 
                 #  instance_mode=ColorMode.IMAGE  # remove the colors of unsegmented pixels. This option is only available for segmentation models
 #   )
  #  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
   # cv2_imshow(out.get_image()[:, :, ::-1])

In [25]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("signs_test", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "signs_test")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[12/15 13:26:44 d2.data.datasets.coco]: Loaded 384 images in COCO format from /Users/Rekha/Documents/road_signs/labels/valid/coco.json
[12/15 13:26:44 d2.data.build]: Distribution of instances among all 15 categories:
|   category    | #instances   |   category    | #instances   |   category    | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
|  red_square   | 2            |  blue_square  | 12           | white_square  | 25           |
| right_of_way  | 15           |   direction   | 68           |  blue_circle  | 31           |
| yellow_blac.. | 18           |     yield     | 31           |  zone_start   | 4            |
|  red_circle   | 57           | red_blue_ci.. | 4            | yield_bakside | 14           |
| blue_white_.. | 22           |   zone_end    | 8            | red_triangle  | 3            |
|               |              |               |              |               |              |
|     total     | 314 

### training sets

In [27]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("signs", )
predictor = DefaultPredictor(cfg)

In [28]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("signs", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "signs")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[12/15 13:38:41 d2.data.datasets.coco]: Loaded 1484 images in COCO format from /Users/Rekha/Documents/road_signs/labels/train/coco.json
[12/15 13:38:41 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/15 13:38:41 d2.data.common]: Serializing 1484 elements to byte tensors and concatenating them all ...
[12/15 13:38:41 d2.data.common]: Serialized dataset takes 0.41 MiB
[12/15 13:38:41 d2.evaluation.evaluator]: Start inference on 1484 batches
[12/15 13:38:42 d2.evaluation.evaluator]: Inference done 11/1484. Dataloading: 0.0520 s/iter. Inference: 0.0518 s/iter. Eval: 0.0043 s/iter. Total: 0.1081 s/iter. ETA=0:02:39
[12/15 13:38:47 d2.evaluation.evaluator]: Inference done 52/1484. Dataloading: 0.0620 s/iter. Inference: 0.0523 s/iter. Eval: 0.0081 s/iter. Total: 0.1225 s/iter. ETA=0:02:55
[12/15 13:38:53 d2.evaluation.evaluator]: Inference done 94/1484. Dataloading: 0.0633 s/

In [3]:
#from detectron2.utils.visualizer import ColorMode
#dataset_dicts = DatasetCatalog.get("signs")
#for d in random.sample(dataset_dicts, 10):    
  #  im = cv2.imread(d["file_name"])
 #   outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
  #  v = Visualizer(im[:, :, ::-1],
                #  metadata=signs_metadata, 
                 #  scale=0.5, 
                  # instance_mode=ColorMode.IMAGE  # remove the colors of unsegmented pixels. This option is only available for segmentation models
  #  )
  #  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
   # cv2_imshow(out.get_image()[:, :, ::-1])